In [1]:
# For data
import networkx as nx
from parse import *
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from IPython.core.display import display, HTML
from IPython.display import Javascript
from string import Template
import json

In [3]:
import os
import pandas as pd

In [4]:
H1=nx.read_graphml('change_topo/cBren.graphml')
H = nx.Graph(H1)
H

In [5]:
len(H.nodes())

37

In [6]:
len(H.edges())

38

In [7]:
# Base for measure is Mb/s
mmnt = {'b/s':1e-6, 'kb/s': 1e-3, 'Mb/s': 1, 'Gb/s':1e+3, 'Tb/s':1e+6}

In [8]:
def unit_measure(strr):
    strr = strr.replace(" ", "")
    i = strr.rfind('b/s') + 3
    strr = strr[:i]
    for m in mmnt.keys():
        if m in strr:
            pattern = '{:d}' + m
            k = parse(pattern, strr)
            if k != None:
                return k[0]*mmnt[m]
    return None

In [9]:
def print_Graph(H):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[0], edge[1])
        perc = data['BackgroundFlow'] / unit_measure(data['LinkLabel'])
        color = 'GREEN'
        if perc <= 0.5:
            color = 'GREEN'
        elif perc <= 0.8:
            color = 'YELLOW'
        elif perc <= 1:
            color = 'RED'
        edges.append({'from': edge[1], 'to': edge[0], 
                      'dashes':False, 'label': (str)(unit_measure(data['LinkLabel'])) + 'Mb/s', 
                      'font':{'align': 'top'},
                      'color': color,
                      'width': 4})
#         else:
#             edges.append({'from': edge[1], 'to': edge[0],
#                           'dashes':True, 'label': '0Mb/s', 
#                           'font':{'align': 'top'},
#                           'color': 'BLACK'})
    return (nodes, edges)

In [10]:
nodes, edges = print_Graph(H)

In [14]:
# Transform the graph into a JSON graph
data = {"nodes":nodes, "edges":edges}
jsonGraph = json.dumps(data, indent=4)

# Send to Javascript
Javascript("""window.jsonGraph={};""".format(jsonGraph))

In [18]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('mynetwork');
    
    var options = {
        width: '900px',
        height: '500px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [37]:
%%html
<div id="mynetwork"></div>

In [54]:
Bandwidth = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))
BackFlows = pd.DataFrame(0, index=np.arange(len(H.nodes())), 
                            columns=np.arange(len(H.nodes())))

In [55]:
for edge in H.edges(data = True):
    
    data_edge = H.get_edge_data(edge[0], edge[1])
    bndw = unit_measure(data_edge['LinkLabel'])
    bflw = data_edge['BackgroundFlow']
#     print(Bandwidth.shape)
#     print(edge)
#     print(Bandwidth[0][237])
    Bandwidth[int(edge[1])][int(edge[0])] = bndw
    Bandwidth[int(edge[0])][int(edge[1])] = bndw
    BackFlows[int(edge[1])][int(edge[0])] = bflw
    BackFlows[int(edge[0])][int(edge[1])] = bflw

In [56]:
Bandwidth.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,300,0,0,0,0,0,0,...,0,200,0,0,0,0,0,0,0,0
1,0,0,0,300,300,0,300,300,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,300,0,0,0,0,0,0,...,0,0,0,0,0,0,0,100,0,0
3,300,300,300,0,0,0,0,0,300,0,...,0,0,100,300,0,0,0,0,0,0
4,0,300,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [57]:
BackFlows.head(5)

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0,0,0,135,0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1,0,0,0,126,96,0,264,264,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,42,0,0,0,0,0,0,...,0,0,0,0,0,0,0,30,0,0
3,135,126,42,0,0,0,0,0,294,0,...,0,0,90,96,0,0,0,0,0,0
4,0,96,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
from itertools import product

def print_answer(status, correct, answ, V, K, M):
    print(status)
    if correct == "correct":
        for i, j, k, t in product(V, V, K, M):
            if (answ[i][j][k][t] != None) and (answ[i][j][k][t] > 0.0):
                print("answer[{}][{}][{}][{}] = {}".format(i, j, k, t, answ[i][j][k][t]))
    else:
        print(correct)

In [59]:
Contracts = [{'volume': 60, 't_s':  60, 't':  60, 'A': 3, 'B': 8},
             {'volume': 100, 't_s':  20, 't':  50, 'A': 7, 'B': 21},
             {'volume': 50, 't_s':  10, 't':  50, 'A': 5, 'B': 26},
             {'volume': 100, 't_s':  0, 't':  50, 'A': 4, 'B': 3},
             {'volume': 100, 't_s':  100, 't':  50, 'A': 9, 'B': 17},
             {'volume': 100, 't_s':  0, 't':  20, 'A': 12, 'B': 19},
             {'volume': 100, 't_s':  50, 't':  50, 'A': 8, 'B': 5},
             {'volume': 100, 't_s':  70, 't':  50, 'A': 26, 'B': 8},
             {'volume': 100, 't_s':  40, 't':  50, 'A': 6, 'B': 30},
             {'volume': 100, 't_s':  30, 't':  50, 'A': 33, 'B': 12}]

In [60]:
def set_time(t0, T, Contracts):
    t = {}
    points = set()
    points.add(t0)
    points.add(T)
    for c in Contracts:
        points.add(c['t_s'])
        points.add(c['t_s'] + c['t'])
    points = sorted(list(points))
    for i in range(len(points) - 1):
        e = points[i + 1] - points[i]
        C = set()
        for c in range(len(Contracts)):
            if points[i] >= Contracts[c]['t_s'] and points[i] < Contracts[c]['t_s'] + Contracts[c]['t']:
                C.add(c)
        t[i] = {'em': e, 'Ctm': C}
    return points, t

In [61]:
mmin = 10000000
mmax = 0
for c in Contracts:
    if c['t_s'] < mmin:
        mmin = c['t_s']
    if c['t_s'] + c['t'] > mmax:
        mmax = c['t_s'] + c['t']

In [62]:
V = set(range(Bandwidth.shape[0])) # число вершин
K = set(range(len(Contracts)))     # количество контрактов К
# t0 = 0                             # начало отсчета t0
# T = 1000                           # конец отсчета в сек
t0 = mmin
T = mmax
timeline, Intervals = set_time(t0, T, Contracts)
M = set(range(len(Intervals)))
timeline, Intervals

([0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 120, 150],
 {0: {'em': 10, 'Ctm': {3, 5}},
  1: {'em': 10, 'Ctm': {2, 3, 5}},
  2: {'em': 10, 'Ctm': {1, 2, 3}},
  3: {'em': 10, 'Ctm': {1, 2, 3, 9}},
  4: {'em': 10, 'Ctm': {1, 2, 3, 8, 9}},
  5: {'em': 10, 'Ctm': {1, 2, 6, 8, 9}},
  6: {'em': 10, 'Ctm': {0, 1, 6, 8, 9}},
  7: {'em': 10, 'Ctm': {0, 6, 7, 8, 9}},
  8: {'em': 10, 'Ctm': {0, 6, 7, 8}},
  9: {'em': 10, 'Ctm': {0, 6, 7}},
  10: {'em': 20, 'Ctm': {0, 4, 7}},
  11: {'em': 30, 'Ctm': {4}}})

In [66]:
import ipynb.fs.full.SolvePyomo as spyomo
import ipynb.fs.full.SolveMIP as mip

Using Python-MIP package version 1.8.0


In [72]:
status, correct_pyomo, answ_pyomo, time_pyomo = spyomo.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_pyomo)
print_answer(status, correct_pyomo, answ_pyomo, V, K, M)

Total time:  28.567155361175537

Problem: 
- Name: unknown
  Lower bound: 278.0
  Upper bound: 278.0
  Number of objectives: 1
  Number of constraints: 632
  Number of variables: 440
  Number of binary variables: 0
  Number of integer variables: 164280
  Number of nonzeros: 440
  Sense: minimize
Solver: 
- Status: ok
  User time: -1.0
  System time: 4.4
  Wallclock time: 5.55
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 5.618938684463501
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

answer[1][3][1][2] = 2.0
answer[1][3][1][3] = 2.0
answer[1][3][1][4] = 2.0
answer[1][3][1][5] = 2.0
answer[1][3][1][6] = 2.0
answer[1][3][3][0] = 2.0
answer[1][3][3][1] = 2.0
answer[1][3][3][2] = 2

In [64]:
def send_toJS(t, K, H, answ):
    nodes = []
    for node in H.nodes():
        nodes.append({'id': node, 'label': node})
    edges = []
    for edge in H.edges():
        data = H.get_edge_data(edge[1], edge[0])
        if data['LinkLabel'] != 'Ethernet':
            k_flow = 0
            label = ''
            for k in K:
                if answ[int(edge[0])][int(edge[1])][k][t] != 0:
                    k_flow += answ[int(edge[0])][int(edge[1])][k][t]
                    label += ', №' + (str)(k)
                if answ[int(edge[1])][int(edge[0])][k][t] != 0:
                    k_flow += answ[int(edge[1])][int(edge[0])][k][t]
                    label += ', №' + (str)(k)
            color = 'BLUE'
            if k_flow != 0:
                perc = (k_flow + BackFlows[int(edge[0])][int(edge[1])]) / Bandwidth[int(edge[0])][int(edge[1])]
                label += "\n" + (str)(k_flow) + "Mb/s"
                if perc <= 0.5:
                    color = 'GREEN'
                elif perc <= 0.8:
                    color = 'YELLOW'
                elif perc <= 1:
                    color = 'RED'        
            edges.append({'from': edge[1], 'to': edge[0], 
                          'dashes':False, 'label': label[2:],
                          'font':{'align': 'top'},
                          'color': color,
                          'width': 4})
        else:
            edges.append({'from': edge[1], 'to': edge[0],
                          'dashes':True, 'label': '0Mb/s', 
                          'font':{'align': 'top'},
                          'color': 'BLACK'})
            
    # Transform the graph into a JSON graph
    data = {"nodes":nodes, "edges":edges}
    jsonGraph = json.dumps(data, indent=4)

    return jsonGraph

In [65]:
t = 0
jsonGraph = send_toJS(t, K, H, answ_pyomo)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

NameError: name 'answ_pyomo' is not defined

In [75]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_pyomo');
    
    var options = {
        width: '1000px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [76]:
%%html
<div id="resultnetwork_pyomo"></div>

In [77]:
status, correct_mip, answ_mip, time_mip = mip.evaluate_model(V, K, M, Contracts, Intervals, Bandwidth, BackFlows)
print("Total time: ", time_mip)
print_answer(status, correct_mip, answ_mip, V, K, M)

Total time:  15.440474271774292
475182
answer[1][3][1][2] = 2.0
answer[1][3][1][3] = 2.0
answer[1][3][1][4] = 2.0
answer[1][3][1][5] = 2.0
answer[1][3][1][6] = 2.0
answer[1][3][3][0] = 2.0
answer[1][3][3][1] = 2.0
answer[1][3][3][2] = 2.0
answer[1][3][3][3] = 2.0
answer[1][3][3][4] = 2.0
answer[1][3][8][4] = 2.0
answer[1][3][8][5] = 2.0
answer[1][3][8][6] = 2.0
answer[1][3][8][7] = 2.0
answer[1][3][8][8] = 2.0
answer[2][3][9][3] = 2.0
answer[2][3][9][4] = 2.0
answer[2][3][9][5] = 2.0
answer[2][3][9][6] = 2.0
answer[2][3][9][7] = 2.0
answer[3][8][0][6] = 1.0
answer[3][8][0][7] = 1.0
answer[3][8][0][8] = 1.0
answer[3][8][0][9] = 1.0
answer[3][8][0][10] = 1.0
answer[3][8][7][7] = 2.0
answer[3][8][7][8] = 2.0
answer[3][8][7][9] = 2.0
answer[3][8][7][10] = 2.0
answer[3][14][6][5] = 2.0
answer[3][14][6][6] = 2.0
answer[3][14][6][7] = 2.0
answer[3][14][6][8] = 2.0
answer[3][14][6][9] = 2.0
answer[3][21][1][2] = 2.0
answer[3][21][1][3] = 2.0
answer[3][21][1][4] = 2.0
answer[3][21][1][5] = 2.0


In [78]:
(answ_mip == answ_pyomo).all()

True

In [79]:
t = 0
jsonGraph = send_toJS(t, K, H, answ_mip)
Javascript("""window.jsonGraph={};""".format(jsonGraph))

<IPython.core.display.Javascript object>

In [186]:
%%javascript
requirejs.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // create a network
    var container = document.getElementById('resultnetwork_mip');
    
    var options = {
        width: '800px',
        height: '400px',
        edges:{
          shadow: true,
          smooth: true,
          background:{
            enabled: true,
            color: '#ff0000'
          }
        }
    }
    
    // We load the JSON graph we generated from iPython input
    var graph = window.jsonGraph;
    
    // Display Graph
    var network = new vis.Network(container, graph, options);
});

<IPython.core.display.Javascript object>

In [187]:
%%html
<div id="resultnetwork_mip"></div>